In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import imageParse as imgP
from audioSent_model import VAD_audio
import audioSent_train as as_train
import imageSent as imgS

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
VAD_map = np.array([
    [1,0.735,0.772], #0: 'happy'
    [0.918,0.61,0.566], #1: 'funny'
    [0.225,0.333,0.149], #2: 'sad'
    [0.63,0.52,0.509], #3: 'tender'
    [0.95,0.792,0.789], #4: 'exciting'
    [0.122,0.83,0.604], #5: 'angry'
    [0.062,0.952,0.528], #6: 'scary'
])

VAD_pd = pd.DataFrame(
    columns = ["valence", "arousal", "dominance"], 
    data = VAD_map, 
    index = ['happy', 'funny', 'sad', 'tender', 
             'exciting', 'angry', 'scary'])

### Audio Sentiment Model

In [4]:
train_context_pd = pd.read_parquet(f"../data/unbal_music_contexts.parquet")
train_embeddings = np.load(f"../data/unbal_music_embeddings.npy")
test_context_pd = pd.read_parquet(f"../data/bal_music_contexts.parquet")
test_embeddings = np.load(f"../data/bal_music_embeddings.npy")

In [5]:
audio_train = tf.cast(train_embeddings.reshape(-1, 10, 128, 1)/128.0, 
                             dtype = tf.float32)
label_train = tf.cast(train_context_pd[["valence", "arousal", "dominance"]].to_numpy(), 
                        dtype = tf.float32)
audio_test = tf.cast(test_embeddings.reshape(-1, 10, 128, 1)/128.0, 
                             dtype = tf.float32)
label_test = tf.cast(test_context_pd[["valence", "arousal", "dominance"]].to_numpy(), 
                        dtype = tf.float32)
seg_label_train = train_context_pd["mood"].to_numpy()
seg_label_test = test_context_pd["mood"].to_numpy()

2022-12-10 16:00:12.727729: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
numEpoch = 4
audio_model = VAD_audio()

as_train.train_full(audio_model, numEpoch, audio_train, label_train)

Epoch: 1
train_loss = 0.1349092572927475 

Epoch: 2
train_loss = 0.10126351565122604 

Epoch: 3
train_loss = 0.08926253020763397 

Epoch: 4
train_loss = 0.07896057516336441 



### Image Sentiment Model

In [7]:
abs_X0, abs_Y0, abs_X1, abs_Y1 = imgP.parseData_Abs()
abs_X0 = imgP.input_prep_fn(abs_X0)
abs_X1 = imgP.input_prep_fn(abs_X1)

In [8]:
art_X0, art_Y0, art_X1, art_Y1 = imgP.parseData_Art()
art_X0 = imgP.input_prep_fn(art_X0)
art_X1 = imgP.input_prep_fn(art_X1)

In [9]:
abs_model = imgS.ImageSentModel(name='abs')
abs_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), VAD_map=VAD_map)
abs_Y0_VAD = VAD_pd.loc[abs_Y0]
abs_Y1_VAD = VAD_pd.loc[abs_Y1]
abs_model.fit(
    (abs_X0, abs_Y0_VAD), abs_Y0_VAD,
    epochs     = 100,
    batch_size = 8,
);

Epoch 1/100
28/28 [==============================] - 1s 11ms/step - mse_loss: 0.6985 - acc: 0.1339
Epoch 2/100
28/28 [==============================] - 0s 11ms/step - mse_loss: 0.6443 - acc: 0.1339
Epoch 3/100
28/28 [==============================] - 0s 10ms/step - mse_loss: 0.6211 - acc: 0.1339
Epoch 4/100
28/28 [==============================] - 0s 12ms/step - mse_loss: 0.6046 - acc: 0.1339
Epoch 5/100
28/28 [==============================] - 0s 11ms/step - mse_loss: 0.5741 - acc: 0.1339
Epoch 6/100
28/28 [==============================] - 0s 11ms/step - mse_loss: 0.5347 - acc: 0.1339
Epoch 7/100
28/28 [==============================] - 0s 12ms/step - mse_loss: 0.5056 - acc: 0.1384
Epoch 8/100
28/28 [==============================] - 0s 12ms/step - mse_loss: 0.4603 - acc: 0.1250
Epoch 9/100
28/28 [==============================] - 0s 13ms/step - mse_loss: 0.4302 - acc: 0.1339
Epoch 10/100
28/28 [==============================] - 0s 11ms/step - mse_loss: 0.4081 - acc: 0.1429
Epoch 11/

28/28 [==============================] - 0s 11ms/step - mse_loss: 0.0053 - acc: 0.3438
Epoch 84/100
28/28 [==============================] - 0s 14ms/step - mse_loss: 0.0063 - acc: 0.3482
Epoch 85/100
28/28 [==============================] - 0s 13ms/step - mse_loss: 0.0114 - acc: 0.3438
Epoch 86/100
28/28 [==============================] - 0s 12ms/step - mse_loss: 0.0092 - acc: 0.3571
Epoch 87/100
28/28 [==============================] - 0s 12ms/step - mse_loss: 0.0055 - acc: 0.3527
Epoch 88/100
28/28 [==============================] - 0s 13ms/step - mse_loss: 0.0043 - acc: 0.3527
Epoch 89/100
28/28 [==============================] - 0s 13ms/step - mse_loss: 0.0047 - acc: 0.3616
Epoch 90/100
28/28 [==============================] - 0s 13ms/step - mse_loss: 0.0042 - acc: 0.3571
Epoch 91/100
28/28 [==============================] - 0s 13ms/step - mse_loss: 0.0043 - acc: 0.3527
Epoch 92/100
28/28 [==============================] - 0s 13ms/step - mse_loss: 0.0038 - acc: 0.3571
Epoch 93/100


In [10]:
art_model = imgS.ImageSentModel(name='art')
art_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), VAD_map=VAD_map)
art_Y0_VAD = VAD_pd.loc[art_Y0]
art_Y1_VAD = VAD_pd.loc[art_Y1]
art_model.fit(
    (art_X0, art_Y0_VAD), art_Y0_VAD,
    epochs     = 100,
    batch_size = 8,
);

Epoch 1/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.7463 - acc: 0.1265
Epoch 2/100
81/81 [==============================] - 1s 11ms/step - mse_loss: 0.7269 - acc: 0.1265
Epoch 3/100
81/81 [==============================] - 1s 13ms/step - mse_loss: 0.7021 - acc: 0.1281
Epoch 4/100
81/81 [==============================] - 1s 13ms/step - mse_loss: 0.6820 - acc: 0.1265
Epoch 5/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.6542 - acc: 0.1281
Epoch 6/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.6148 - acc: 0.1281
Epoch 7/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.5734 - acc: 0.1281
Epoch 8/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.5344 - acc: 0.1265
Epoch 9/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.4807 - acc: 0.1265
Epoch 10/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.4396 - acc: 0.1265
Epoch 11/

81/81 [==============================] - 1s 10ms/step - mse_loss: 0.0266 - acc: 0.1852
Epoch 84/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.0255 - acc: 0.1944
Epoch 85/100
81/81 [==============================] - 1s 9ms/step - mse_loss: 0.0240 - acc: 0.1867
Epoch 86/100
81/81 [==============================] - 1s 9ms/step - mse_loss: 0.0252 - acc: 0.1960
Epoch 87/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.0248 - acc: 0.1883
Epoch 88/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.0245 - acc: 0.1914
Epoch 89/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.0255 - acc: 0.1852
Epoch 90/100
81/81 [==============================] - 1s 10ms/step - mse_loss: 0.0238 - acc: 0.1898
Epoch 91/100
81/81 [==============================] - 1s 9ms/step - mse_loss: 0.0243 - acc: 0.1836
Epoch 92/100
81/81 [==============================] - 1s 9ms/step - mse_loss: 0.0266 - acc: 0.1898
Epoch 93/100
81/8

### Matching Images to Audio

In [11]:
classes_to_sentiments = {
    276:'happy',
    277:'funny',
    278:'sad',
    279:'tender',
    280:'exciting',
    281:'angry',
    282:'scary'
}
audio_Y1 = [classes_to_sentiments[_] for _ in seg_label_test]

In [12]:
audio_pred_VAD = audio_model(audio_test)

In [13]:
abs_pred_VAD = abs_model((abs_X1, abs_Y1_VAD))

In [14]:
art_pred_VAD = art_model((art_X1, art_Y1_VAD))

In [15]:
abs_matches = imgS.getClosest(audio_pred_VAD, abs_pred_VAD)
abs_matches = [(1.0 if abs_Y1[_] == audio_Y1[abs_matches[_]] else 0.0) for _ in range(len(abs_matches))]
abs_acc = tf.math.reduce_mean(tf.constant(abs_matches))
print(abs_acc.numpy())

0.071428575


In [16]:
art_matches = imgS.getClosest(audio_pred_VAD, art_pred_VAD)
art_matches = [(1.0 if art_Y1[_] == audio_Y1[art_matches[_]] else 0.0) for _ in range(len(art_matches))]
art_acc = tf.math.reduce_mean(tf.constant(art_matches))
print(art_acc.numpy())

0.19254659
